# Importazione librerie

In [4]:
import re
import spacy
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer
print("Importazione librerie completata")

Importazione librerie completata


# Pipeline social

In [5]:
class PipelineSocial:
    def __init__(self, lingua='italian'):
        """Pipeline NLP adattata a testi social (emoji, slang, hashtag)"""
        self.lingua = lingua
        self.nlp = spacy.load('it_core_news_sm' if lingua == 'italian' else 'en_core_web_sm')
        self.stemmer = SnowballStemmer(lingua)
        self.stop_words = set(stopwords.words(lingua))

        # Dizionario di sostituzioni per emoji e slang comuni
        self.mappa_semantica = {
            "😱": "sorpresa",
            "🔥": "fantastico",
            "😂": "divertente",
            "😎": "cool",
            "😍": "adorabile",
            "lol": "divertente",
            "omg": "incredibile",
            "wtf": "assurdo",
            "top": "eccellente"
        }

    def normalizza_testo(self, testo):
        """Normalizzazione semantica (emoji, hashtag, slang, lowercasing)"""
        testo = testo.lower()

        # Sostituisci emoji e slang tramite dizionario
        for chiave, sostituto in self.mappa_semantica.items():
            testo = testo.replace(chiave, f" {sostituto} ")

        # Espandi hashtag (mantieni la parola chiave)
        testo = re.sub(r'#(\w+)', r'\1', testo)

        # Rimuovi mention tipo @username
        testo = re.sub(r'@\w+', '', testo)

        # Rimuovi URL ed email
        testo = re.sub(r'http\S+|www\S+|\S+@\S+', '', testo)

        # Rimuovi caratteri non alfanumerici (tranne apostrofi e spazi)
        testo = re.sub(r"[^a-zàèéìòù'\s]", ' ', testo)

        # Rimuovi spazi multipli
        testo = re.sub(r'\s+', ' ', testo).strip()
        return testo

    def tokenizza_e_lemmatizza(self, testo):
        """Tokenizzazione + Lemmatizzazione"""
        doc = self.nlp(testo)
        tokens = []
        for token in doc:
            if not token.is_punct and not token.is_space and len(token.text) > 2:
                if token.lemma_ != '-PRON-':
                    tokens.append(token.lemma_)
                else:
                    tokens.append(self.stemmer.stem(token.text))
        return tokens

    def rimuovi_stop_words(self, tokens):
        """Rimuove stopwords mantenendo parole chiave"""
        return [t for t in tokens if t not in self.stop_words]

    def processa_testo(self, testo):
        """Esegue tutti gli step"""
        testo_norm = self.normalizza_testo(testo)
        tokens = self.tokenizza_e_lemmatizza(testo_norm)
        tokens = self.rimuovi_stop_words(tokens)
        return tokens

# Test

In [6]:
# Test del caso specifico
testo = "OMG 😱 non ci credo, la nuova stagione di Stranger Things è PAZZESCA!!! 🔥🔥 Gli attori sempre top, ma la trama stavolta è assurda lol 😂 #Netflix #strangerthings"

pipeline = PipelineSocial()
risultato = pipeline.processa_testo(testo)

print("🔤 Tokens finali:", risultato)

🔤 Tokens finali: ['incredibile', 'sorpreso', 'credere', 'nuovo', 'stagione', 'strangere', 'things', 'pazzesca', 'fantastico', 'fantastico', 'attore', 'sempre', 'eccellente', 'trama', 'stavolta', 'assurdo', 'divertente', 'divertente', 'netflix', 'strangerthings']


# Approccio utilizzato
- Creazione di un dizionario per tradurre slang ed emoji comuni in parole chiave descrittive;
- Sostituituzione degli hashtag/mention preservando termini utili (es. #Netflix → netflix);
- Pulizia più robusta per emoji e simboli ripetuti;
- Mantenimento dell’approccio spaCy + NLTK per tokenizzazione, lemmatizzazione e stopwords.